In [1]:
import csv

import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from tensorflow import keras

from functions.image import get_filtered_satellite_float32, get_satellite_raw_uint8

In [2]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)
block_size = 30


def get_image(coord):
    return get_filtered_satellite_float32(coord, block_size, img_data, dataset)

In [3]:
with open('training/data_bin.csv', 'r') as f:
    reader = csv.DictReader(f)
    labeled_coords = [r for r in reader]
print("labeled dataset contains {} points".format(len(labeled_coords)))

labeled dataset contains 115 points


In [4]:
X = list(map(get_image, labeled_coords))
Y = [int(l['windmill']) for l in labeled_coords]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [6]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2021-12-14 11:05:59.749093: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 11:05:59.758638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 11:05:59.759202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 11:05:59.759951: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10


2021-12-14 11:06:02.158709: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2021-12-14 11:06:02.696169: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


3/3 [==============================] - 2s 20ms/step - loss: 0.6678 - accuracy: 0.6750
Epoch 2/10
3/3 [==============================] - 0s 4ms/step - loss: 0.5930 - accuracy: 0.6750
Epoch 3/10
3/3 [==============================] - 0s 4ms/step - loss: 0.5349 - accuracy: 0.6500
Epoch 4/10
3/3 [==============================] - 0s 4ms/step - loss: 0.4941 - accuracy: 0.6625
Epoch 5/10
3/3 [==============================] - 0s 3ms/step - loss: 0.4507 - accuracy: 0.7250
Epoch 6/10
3/3 [==============================] - 0s 4ms/step - loss: 0.4197 - accuracy: 0.7875
Epoch 7/10
3/3 [==============================] - 0s 3ms/step - loss: 0.3588 - accuracy: 0.8875
Epoch 8/10
3/3 [==============================] - 0s 7ms/step - loss: 0.3435 - accuracy: 0.8625
Epoch 9/10
3/3 [==============================] - 0s 6ms/step - loss: 0.2928 - accuracy: 0.8750
Epoch 10/10
3/3 [==============================] - 0s 3ms/step - loss: 0.2886 - accuracy: 0.9000


In [8]:
model.evaluate(X_test, Y_test, verbose=2)

2/2 - 0s - loss: 0.2202 - accuracy: 0.8857 - 130ms/epoch - 65ms/step


[0.22021642327308655, 0.8857142925262451]

In [9]:
dataset = rasterio.open("./data/crs-2.tiff")
img_data = dataset.read(1)
block_size = 30


def get_image(coord):
    return get_filtered_satellite_float32(coord, block_size, img_data, dataset)

In [23]:
left_top_coord = (7.67337,53.79955)  # enter coordinate here
img_h, img_w = (750, 500)
left_top_x, left_top_y = dataset.index(left_top_coord[0], left_top_coord[1])
needles = []
step_size = 8

for xx in range(left_top_x + block_size // 2, left_top_x + img_w - block_size // 2, step_size):
    for yy in range(left_top_y + block_size // 2, left_top_y + img_h - block_size // 2, step_size):
        cc = dataset.transform * (yy, xx)
        needles.append({'lon': cc[0], 'lat': cc[1]})

needle_data = np.stack([get_image(coord) for coord in needles])
results = model.predict(needle_data)

In [24]:
from PIL import Image

j = 0

for i, result in enumerate(results):
    if result[1] < result[0]:
        continue
    coord = needles[i]
    a = get_satellite_raw_uint8(coord, block_size, img_data, dataset)
    im = Image.fromarray(a)
    im.save("./examples/%s-%s.png" % (str(coord["lon"]).replace(".", "_"), str(coord["lat"]).replace(".", "_")))
    if j > 200:
        break
    j += 1